In [15]:
import h5py
import numpy as np
from sklearn.preprocessing import StandardScaler

In [16]:
ef = h5py.File('/fast_scratch/QVAE/data/calo/eplus.hdf5','r')
gf = h5py.File('/fast_scratch/QVAE/data/calo/gamma.hdf5','r')
pf = h5py.File('/fast_scratch/QVAE/data/calo/piplus.hdf5','r')

In [17]:
hdfs = [ef, gf, pf]
nplcats = []

for hdf in hdfs:
    npl0 = np.array(hdf['layer_0'])
    npl1 = np.array(hdf['layer_1'])
    npl2 = np.array(hdf['layer_2'])
    
    npl0 = npl0.reshape(npl0.shape[0], -1)
    npl1 = npl1.reshape(npl1.shape[0], -1)
    npl2 = npl2.reshape(npl2.shape[0], -1)
    
    nplcats.append(np.concatenate([npl0, npl1, npl2], axis=1))
print(np.shape(hdfs[0]))

(5,)


In [20]:
nplcatscaled = []
transformers = []
arrmins = [[], [], []]
epsilon = 1e-2

for i in range(len(nplcats)):
    nparr = nplcats[i]
    nparr = np.where(nparr > 0., nparr, np.nan)
    transformer = StandardScaler().fit(nparr)
    nparr = transformer.transform(nparr)
    transformers.append(transformer)
    
    nparr = np.where(np.isnan(nparr), np.inf, nparr)
    
    for j in range(nparr.shape[1]):
        arrmin = np.amin(nparr[:, j])
        
        if arrmin < 0 and not np.isnan(arrmin):
            nparr[:, j] -= arrmin
            nparr[:, j] += epsilon
            arrmins[i].append(arrmin)
        else:
            arrmins[i].append(0.)
            
    nparr = np.where(np.isinf(nparr), 0, nparr)
    
    for j in range(nparr.shape[1]):
        arrmin = np.amin(nparr[:, j])
        if arrmin < 0:
            print(j, arrmin)
            
    nplcatscaled.append(nparr)

<KeysViewHDF5 ['energy', 'layer_0', 'layer_1', 'layer_2', 'overflow']>


In [19]:
ef_scaled = h5py.File('/fast_scratch/QVAE/data/calo_scaled/eplus.hdf5','w')
gf_scaled = h5py.File('/fast_scratch/QVAE/data/calo_scaled/gamma.hdf5','w')
pf_scaled = h5py.File('/fast_scratch/QVAE/data/calo_scaled/piplus.hdf5','w')

PermissionError: [Errno 13] Unable to create file (unable to open file: name = '/fast_scratch/QVAE/data/calo_scaled/eplus.hdf5', errno = 13, error message = 'Permission denied', flags = 13, o_flags = 242)

In [22]:
hdfs_scaled = [ef_scaled, gf_scaled, pf_scaled]
layer_shapes = {}
for key in hdf.keys():
    if key == "energy" or key == "overflow":
        pass
    else:
        layer_shapes[key] = hdf[key].shape

NameError: name 'ef_scaled' is not defined

In [ ]:
layer_shapes

In [21]:
for hdf, hdf_scaled, scaled_data in zip(hdfs, hdfs_scaled, nplcatscaled):
    offset = 0
    for key in hdf.keys():
        if key == "energy" or key == "overflow":
            hdf_scaled.create_dataset(key, data=hdf[key])
        else:
            layer_shape = layer_shapes[key]
            print(scaled_data.shape)
            layer_data = scaled_data[:, offset:offset+(layer_shape[1]*layer_shape[2])]
            print(layer_data.shape)
            layer_data = layer_data.reshape(layer_shape)
            hdf_scaled.create_dataset(key, data=layer_data)
            offset += layer_shape[1]*layer_shape[2]
print(hdfs[0])

NameError: name 'hdfs_scaled' is not defined

In [ ]:
for hdf_scaled in hdfs_scaled:
    for key in hdf_scaled.keys():
        print(key, hdf_scaled[key].shape, hdf_scaled[key].dtype)

In [ ]:
for hdf_scaled in hdfs_scaled:
    hdf_scaled.close()

In [ ]:
for nplcat in nplcatscaled:
    print(nplcat.shape)

In [ ]:
print(len(arrmins[0]))

In [7]:
nplcatinv = []

for i in range(len(nplcatscaled)):
    nparr = nplcatscaled[i]
    nparr = np.where(nparr > 0., nparr, np.nan)
    
    for j in range(nparr.shape[1]):
        arrmin = arrmins[i][j]
        if arrmin < 0. and not np.isnan(arrmin):
            nparr[:, j] -= epsilon
            nparr[:, j] += arrmin
            
    transformer = transformers[i]
    nparr = transformer.inverse_transform(nparr)
    
    nparr = np.where(np.isinf(nparr), 0, nparr)
    nplcatinv.append(nparr)

In [8]:
for i in range(len(nplcatinv)):
    nparrorig = nplcats[i]
    nparrinv = nplcatinv[i]
    
    for j in range(nparrorig.shape[1]):
        diff = np.sum(nparrorig[:, j] - nparrinv[:, j])
        if diff > 0:
            print(i, j, diff)

0 143 1.3236745033395891e-09
0 144 3.7318459433777207e-10
0 353 4.4510706231903896e-10
0 354 1.8765078380056366e-10
0 365 3.5996983172026376e-10
0 366 3.0881430745921534e-10
1 354 9.078098273107571e-10
1 365 7.495437603921573e-11
1 366 1.301929231090071e-09


In [9]:
params = transformers[0].get_params()

In [10]:
params

{'copy': True, 'with_mean': True, 'with_std': True}

In [11]:
import joblib

In [12]:
joblib.dump(transformers[0], 'scaler.gz')
transformer = joblib.load('scaler.gz')

In [13]:
print(len(transformers))
print(transformers)

3
[StandardScaler(), StandardScaler(), StandardScaler()]


In [ ]:
joblib.dump(transformers[0], '/fast_scratch/QVAE/data/calo_scaled/eplus_scaler.gz')
joblib.dump(transformers[1], '/fast_scratch/QVAE/data/calo_scaled/gamma_scaler.gz')
joblib.dump(transformers[2], '/fast_scratch/QVAE/data/calo_scaled/piplus_scaler.gz')

In [ ]:
ld_transformers = []
ld_transformers.append(joblib.load('/fast_scratch/QVAE/data/calo_scaled/eplus_scaler.gz'))
ld_transformers.append(joblib.load('/fast_scratch/QVAE/data/calo_scaled/gamma_scaler.gz'))
ld_transformers.append(joblib.load('/fast_scratch/QVAE/data/calo_scaled/piplus_scaler.gz'))

In [ ]:
nplcatinv = []

for i in range(len(nplcatscaled)):
    nparr = nplcatscaled[i]
    nparr = np.where(nparr > 0., nparr, np.inf)
    
    for j in range(nparr.shape[1]):
        arrmin = arrmins[i][j]
        if arrmin < 0. and not np.isnan(arrmin):
            nparr[:, j] -= epsilon
            nparr[:, j] += arrmin
            
    transformer = ld_transformers[i]
    nparr = transformer.inverse_transform(nparr)
    
    nparr = np.where(np.isnan(nparr), 0, nparr)
    nplcatinv.append(nparr)

In [ ]:
for i in range(len(nplcatinv)):
    nparrorig = nplcats[i]
    nparrinv = nplcatinv[i]
    
    for j in range(nparrorig.shape[1]):
        diff = np.sum(nparrorig[:, j] - nparrinv[:, j])
        if diff > 1e-4:
            print(i, j, diff)

In [ ]:
print(len(arrmins))

In [ ]:
print(arrmins[0])

In [ ]:
for i, ptype in enumerate(["eplus", "gamma", "piplus"]):
    filepath = "/fast_scratch/QVAE/data/calo_scaled/" + ptype + "_amin.npy"
    with open(filepath, 'wb') as f:
        np.save(f, arrmins[i])

In [ ]:
def method(idx, a=10):
    return a

In [ ]:
method(1)

In [ ]:
a = np.random.randn(50)

In [ ]:
len(a[:50])

In [ ]:
a, b = [50, 10]

In [ ]:
a

In [ ]:
b

In [ ]:
np.logspace(, 10, 100)

In [ ]:
np.log10(0)

In [ ]:
5^/2

In [ ]:
a = [1, 2]
b = [3, 4]

In [ ]:
c = [np.divide(ai, bi) for ai, bi in zip(a, b)]

In [ ]:
c

In [ ]:
a = {"a":1, "b":2}

In [ ]:
a[[a,b]]